# 2

In [ ]:
import IPython.display as ipd
import librosa
import librosa.display
import numpy as np
import pandas as pd

filename = "sound.wav"
data,sample_rate = librosa.load(filename)
ipd.Audio(filename)

Data Check

In [ ]:
def mse(func):
    error=np.abs(data-func())
    return np.mean(error)*1500

def func(cf) :
    time = np.linspace(0, 0.5, len(data))
    sound = np.sum(cf[0]*np.sin(np.array( [i*time for i in cf[1:]] )), axis=0)
    
    return sound

def test_1():
    return func(init_cf)

def test_2():
    return func(gen_list)  

In [ ]:
def generate_list():
    gen_list = [max(data)/4, 0,0,0,0]
    for i in range (1, len(gen_list)):
        gen_list[i] = random.randint(1, 10000)
    
    return gen_list

In [ ]:
init_cf = [max(data)/4, 800, 1000, 1200, 1400]
minimum_cf = [max(data)/4, 800, 1000, 1200, 1400]
minimum_loss = 10000

while minimum_loss > 5:

    for _ in range(10000):
        init_cf[4] = init_cf[4] + 1
        if minimum_loss > mse(test_1):
            print("change")
            print("before : ", minimum_loss)
            print("after : ", mse(test_1))
            print("{} -> {}".format(minimum_cf[1:], init_cf[1:]))
            print("")
            minimum_loss = mse(test_1)
            minimum_cf = init_cf

        for _ in range (10000):
            init_cf[3] = init_cf[3] + 1
            if minimum_loss > mse(test_1):
                print("change")
                print("before : ", minimum_loss)
                print("after : ", mse(test_1))
                print("{} -> {}".format(minimum_cf[1:], init_cf[1:]))
                print("")                        
                minimum_loss = mse(test_1)
                minimum_cf = init_cf
            
            for _ in range (10000):
                    init_cf[2] = init_cf[2] + 1
                    if minimum_loss > mse(test_1):
                        print("change")
                        print("before : ", minimum_loss)
                        print("after : ", mse(test_1))
                        print("{} -> {}".format(minimum_cf[1:], init_cf[1:]))
                        print("")                        
                        minimum_loss = mse(test_1)
                        minimum_cf = init_cf
                    
                    for _ in range (10000):
                        init_cf[1] = init_cf[1] + 1
                        if minimum_loss > mse(test_1):
                            print("change")
                            print("before : ", minimum_loss)
                            print("after : ", mse(test_1))
                            print("{} -> {}".format(minimum_cf[1:], init_cf[1:]))
                            print("")                        
                            minimum_loss = mse(test_1)
                            minimum_cf = init_cf


In [ ]:
import random 

check_list = list()
minimum_loss_cf = [0,0,0,0,0]
minimum_loss = 10000

while minimum_loss > 5:
    gen_list = generate_list()

    while gen_list in check_list:
        gen_list = generate_list()
        print('regenerate')
    
    check_list.append(gen_list)
    
    if minimum_loss > mse(test_2):
        print("Old Frequency: ", minimum_loss_cf[1:])
        print("Old Loss: ", minimum_loss)
        minimum_loss = mse(test_2)
        minimum_loss_cf = gen_list 
        print("Update Frequency: ", minimum_loss_cf[1:])
        print("Update Loss: ",minimum_loss) 
        print("")

# 3

setting

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

# install
## numpy
## matplotlib

In [ ]:
from dataset.mnist import load_mnist

(train_raw_img, train_label), (test_raw_img, test_label) = load_mnist(flatten=False, normalize=False)
print(train_raw_img.shape)
print(train_label.shape)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_raw_img[i][0], cmap=plt.cm.binary)
plt.show()

In [ ]:
np.unique(train_label, return_counts=True)

In [ ]:
# train_dataset split according to the number

new_train_img = [[] for _ in range(10)]
new_train_label = [[] for _ in range(10)]

for i in range(len(train_label)) :
    new_train_img[train_label[i]].append(train_raw_img[i])
    new_train_label[train_label[i]].append(train_label[i])

# print(len(new_train_img[0])) # 0에 해당하는 image 개수
# print(new_train_img[0][0].shape) # 0에 해당하는 image중 첫번째 image의 shape

### Logistic Regression

In [ ]:
def make_sample(idx) :
    sample_img = []
    sample_label = []
    
    # data sampling 
    for i in range(10) :
        if i == idx :
            sample_img += new_train_img[i][:1000]
            sample_label += (new_train_label[i][:1000])
        else :
            sample_img += new_train_img[i][:111]
            sample_label += (new_train_label[i][:111])

    sample_img = np.array(sample_img)
    sample_label = np.array(sample_label)
    
    # normalization (set value 0 ~ 1)
    sample_img = sample_img.astype('float')/255
    
    # target number는 1, 아니면 0
    sample_label = np.where(sample_label==idx, 1 ,0)
    
    # reshape
    sample_img = sample_img.reshape(len(sample_img.squeeze()), -1)
    sample_label = sample_label.reshape(len(sample_label.squeeze()), -1)
    
    return sample_img, sample_label

In [ ]:
train_X, train_y = make_sample(idx = 0) # idx = target number
train_X = np.insert(train_X, 0, 1, axis=1) # bias 추가

# print(train_y.shape)

In [ ]:
# cross entropy loss
def CrossEntropyLoss(w, X, y) :
    delta = 1e-7
    
    preds = 1 / (1+np.exp(-X.dot(w)))
    loss = np.sum(-y*np.log(preds+delta)-(1-y)*np.log(1-preds+delta))/len(preds)
        
    return loss , preds

In [ ]:
# eval (accuracy)

def eval(idx, w) :
    """_summary_

    Args:
        idx : target_number
        w : parameter
    """
    test_X = test_raw_img.astype('float')/255    
    test_X = test_X.reshape(len(test_X.squeeze()), -1)
    test_X = np.insert(test_X, 0, 1, axis=1) # + bias

    test_y = np.where(test_label==idx, 1 ,0)
    test_y = test_y.reshape(len(test_y.squeeze()), -1)
    
    preds = 1/(1+np.exp(-test_X.dot(w)))
    result = np.where(preds>0.5, 1, 0)
    
    acc = np.sum(np.where(result==test_y, True, False))/len(preds)
    
    return acc
    # print('accuracy : ', acc)

In [ ]:
# train

def train(X, y, index) :

    w = np.random.randn(len(X[0]), 1)
    lr = 0.1 # learning rate(수정 가능)
    step = 0
    acc = 0
    prev_loss = float('inf')
    J_history = list()
    ACC_history = list()
    
    while (acc <= 0.85) :
        step += 1
        correct = 0
        
        acc = eval(index, w)
        
        loss, preds = CrossEntropyLoss(w, X, y)
        
        diff = preds - y
        gradient = X.T.dot(diff) / X.shape[0]
        w -= lr * gradient
        
        if abs(loss - prev_loss) < 1e-4:
            break
        
        prev_loss = loss
        J_history.append(loss)
        ACC_history.append(acc)
        print("total step : %d " % step)
        print("error : %f, accuarcy : %f" % (loss, acc))
        
    return w, J_history, ACC_history

In [ ]:
# save weight
w,J_history, ACC_history = train(train_X, train_y, 0)

In [ ]:
# Plot the cost function vs. iterations to check for convergence
plt.plot(J_history, label='Cost Function')
plt.plot(ACC_history, label='Accuracy')
plt.xlabel('Iterations')
plt.ylabel('Cost Function')
plt.title('Gradient Descent Convergence')
plt.show()

### multi class single label classification (using logistic regression)

In [ ]:
train_0X, train_0y = make_sample(idx = 0) # idx = target number
train_0X = np.insert(train_0X, 0, 1, axis=1) # bias 추가
w0,J0_history, ACC0_history = train(train_X, train_y,0)


In [ ]:
train_1X, train_1y = make_sample(idx = 1) # idx = target number
train_1X = np.insert(train_1X, 0, 1, axis=1) # bias 추가
w1,J1_history, ACC1_history = train(train_1X, train_1y,1)

In [ ]:
train_2X, train_2y = make_sample(idx = 2) # idx = target number
train_2X = np.insert(train_2X, 0, 1, axis=1) # bias 추가
w2,J2_history, ACC2_history  = train(train_2X, train_2y,2)

In [ ]:
train_3X, train_3y = make_sample(idx = 3) # idx = target number
train_3X = np.insert(train_3X, 0, 1, axis=1) # bias 추가
w3,J3_history, ACC3_history = train(train_3X, train_3y,3)

In [ ]:
train_4X, train_4y = make_sample(idx = 4) # idx = target number
train_4X = np.insert(train_4X, 0, 1, axis=1) # bias 추가
w4,J4_history, ACC4_history = train(train_4X, train_4y,4)

In [ ]:
train_5X, train_5y = make_sample(idx = 5) # idx = target number
train_5X = np.insert(train_5X, 0, 1, axis=1) # bias 추가
w5,J5_history, ACC5_history = train(train_5X, train_5y,5)

In [ ]:
train_6X, train_6y = make_sample(idx = 6) # idx = target number
train_6X = np.insert(train_6X, 0, 1, axis=1) # bias 추가
w6,J6_history, ACC6_history = train(train_6X, train_6y,6)

In [ ]:

train_7X, train_7y = make_sample(idx = 7) # idx = target number
train_7X = np.insert(train_7X, 0, 1, axis=1) # bias 추가
w7,J7_history, ACC7_history = train(train_7X, train_7y,7)

In [ ]:
train_8X, train_8y = make_sample(idx = 8) # idx = target number
train_8X = np.insert(train_8X, 0, 1, axis=1) # bias 추가
w8,J8_history, ACC8_history = train(train_8X, train_8y,8)

In [ ]:
train_9X, train_9y = make_sample(idx = 9) # idx = target number
train_9X = np.insert(train_9X, 0, 1, axis=1) # bias 추가
w9,J9_history, ACC9_history = train(train_9X, train_9y,9)

In [ ]:
def mult_cls_classifier(X,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9):
    
    res_0 = 1 / (1+np.exp(-X.dot(w0)))
    res_1 = 1 / (1+np.exp(-X.dot(w1)))
    res_2 = 1 / (1+np.exp(-X.dot(w2)))
    res_3 = 1 / (1+np.exp(-X.dot(w3)))
    res_4 = 1 / (1+np.exp(-X.dot(w4)))
    res_5 = 1 / (1+np.exp(-X.dot(w5)))
    res_6 = 1 / (1+np.exp(-X.dot(w6)))
    res_7 = 1 / (1+np.exp(-X.dot(w7)))
    res_8 = 1 / (1+np.exp(-X.dot(w8)))
    res_9 = 1 / (1+np.exp(-X.dot(w9)))
    
    res = np.concatenate((res_0,
                        res_1,
                        res_2,
                        res_3,
                        res_4,
                        res_5,
                        res_6,
                        res_7,
                        res_8,
                        res_9), axis = 1)
    
    return res

In [ ]:
# eval
import pandas as pd

test_X = test_raw_img.astype('float')/255    
test_X = test_X.reshape(len(test_X.squeeze()), -1)
test_X = np.insert(test_X, 0, 1, axis=1) # + bias
print('shape of test_X : ', test_X.shape)
print('shape of test_label : ', test_label.shape)

probs = mult_cls_classifier(test_X,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9)
# make prediction using argmax
max_pred = np.argmax(probs, axis = 1)

acc = np.sum(np.where(test_label==max_pred, True, False))/len(test_X)
print('accuracy : ', acc)

### L2 Regularization with Logistic regression

In [ ]:
# cross entropy loss
def CrossEntropyLoss(w, X, y, alpha) :
    delta = 1e-7
    
    preds = 1 / (1+np.exp(-X.dot(w)))
    reg_term = 0.5 * alpha * np.sum(w**2)
    loss = np.sum(-y*np.log(preds+delta)-(1-y)*np.log(1-preds+delta))/len(preds) + reg_term
        
    return loss , preds

In [ ]:
# train

def train(X, y, index,alpha) :
    """_summary_

    Args:
        X : train_X
        y : train_y

    Returns:
        w : weight
    """
    w = np.random.randn(len(X[0]), 1)
    lr = 0.1 # learning rate(수정 가능)
    step = 0
    acc = 0
    prev_loss = float('inf')    
    weight_capacity = list()
    
    while (acc <= 0.85) :
        step += 1
        correct = 0
        
        acc = eval(index, w)
        
        loss, preds = CrossEntropyLoss(w, X, y, alpha)
        
        diff = preds - y
        gradient = X.T.dot(diff) / X.shape[0]
        weight_capacity.append(np.linalg.norm(w[0][0]))
        w -= lr * gradient
        
        if abs(loss - prev_loss) < 1e-4:
            break
        
        prev_loss = loss
        
        # print("total step : %d " % step)
        # print("error : %f, accuarcy : %f" % (loss, acc))
        
    return w, weight_capacity

In [ ]:
lambdas = [0.01, 0.1, 1.0, 10.0, 100.0]
weights = np.zeros((5, len(lambdas)))

In [ ]:
for i, l in enumerate(lambdas):
    w,_ = train(train_7X, train_7y, 7, l)
    weights[:, i] = np.sum(np.array(w[:1]))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for i in range(5):
    ax.plot(lambdas, weights[i], label=f'Feature {i+1}')
ax.set_xscale('log')
ax.set_xlabel('lambda')
ax.set_ylabel('weight')
ax.legend()
plt.show()

# 4

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

# install
## numpy
## matplotlib

data load & preprocessing

In [ ]:
from dataset.mnist import load_mnist

(train_raw_img, train_label), (test_raw_img, test_label) = load_mnist(flatten=False, normalize=False)
print(train_raw_img.shape)

In [ ]:
# preprocessing (train & inference)

train_img = train_raw_img.reshape(len(train_raw_img.squeeze()), -1)
train_label = train_label.reshape(len(train_label), -1)

test_img = test_raw_img.reshape(len(test_raw_img.squeeze()), -1)
test_label = test_label.reshape(len(test_label), -1)

print(train_img.shape)
print(train_label.shape)
print(test_img.shape)
print(test_label.shape)

In [ ]:
# normalization (set value 0 ~ 1)

train_img = train_img.astype('float')
train_img = train_img/255

test_img = test_img.astype('float')
test_img = test_img/255

### Activation function model

In [ ]:
class Linear :
    def __init__(self, input_size=1, hidden_size=1) :
        self.W = np.random.randn(input_size, hidden_size)
        self.b = np.zeros(hidden_size)
        self.x = None
        self.dW = None
        self.db =  None
    
    def forward(self, x) :
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout, lr) :
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        self.W -= lr*self.dW
        self.b -= lr*self.db
        return dx

In [ ]:
class Relu :
    def __init__(self) :
        self.mask = None
    
    def forward(self, x) :
        self.mask = (x < 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout) :
        dout[self.mask] = 0
        return dout                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
class softmax_with_crossEntropy :
    def __init__(self) :
        self.delta = 1e-7
        # softmax
        self.softmax_x = None
        self.softmax_out = None
        # crossEntropy
        self.pred = None
        self.target = None
        
    def softmax_forward(self, x) :
        self.softmax_x = x
        # subtracting the maximum value for numerical stability
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        self.softmax_out = exp_x / np.sum(exp_x, axis=-1, keepdims=True)
        return self.softmax_out

    def crossEntropy_forward(self, pred, target) :
        self.pred = pred
        self.target = target
        # avoid log(0) by adding delta
        loss = -np.sum(target * np.log(pred + self.delta))
        return loss
    
    
    def backward(self):
        batch_size = self.pred.shape[0]
        dx = (self.pred - self.target) / batch_size
        return dx

Train

In [ ]:
# one_hot label 만드는 함수

def make_one_hot(labels) :
    a = []
    for label in labels :
        one_hot = np.zeros(10)
        one_hot[label] = 1
        a.append(one_hot)
    a = np.array(a)
    return a

# one_hot_labels = make_one_hot(train_label)
# print(train_label[0])
# print(one_hot_labels[0])

In [ ]:
# train version 1

from collections import OrderedDict

def train_MLP(config) :
    lr, num_epoch = config['learning_rate'], config['num_epoch']
    print_loss_interval = 1
    
    layer1 = Linear(784, 100)
    relu = Relu()
    layer2 = Linear(100, 10)
    softmax_with_CE = softmax_with_crossEntropy()
    
    iter_lst = list()
    loss_lst = list()
    Non_weight_lsit = list()
    
    for iter in range(num_epoch) :
        # forward
        x = layer1.forward(train_img)
        x = relu.forward(x)
        x = layer2.forward(x)
        preds = softmax_with_CE.softmax_forward(x)
        
        # loss
        one_hot_labels = make_one_hot(train_label)
        losses = softmax_with_CE.crossEntropy_forward(preds, one_hot_labels)
        loss = losses.sum()/len(preds)
        
        if iter % print_loss_interval == 0:
            print("[epoch %d / %d] average loss : %f" % (iter, num_epoch, loss))
            
        iter_lst.append(iter)
        loss_lst.append(loss)
        
        # backward
        dL = softmax_with_CE.backward()
        Non_weight_lsit.append(np.sum(dL))
        dL = layer2.backward(dL, lr)
        dL = relu.backward(dL)
        dL = layer1.backward(dL, lr)
                     
    model = OrderedDict()
    model['layer1'] = layer1
    model['relu'] = relu
    model['layer2'] = layer2
    model['softmax_with_CE'] = softmax_with_CE
    
    return model,iter_lst, loss_lst,Non_weight_lsit

In [ ]:
config = { 'learning_rate' : 0.1,
            'num_epoch' : 100
          }

model,iter_lst, loss_lst,Non_weight_lsit = train_MLP(config)

In [ ]:
plt.plot(iter_lst, loss_lst)
plt.title("Loss vs. Iteration")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

In [ ]:
def eval(model, train_version = True) :
    if train_version :
        x = train_img
        labels = train_label.squeeze()
        print('In train dataset ... ')
    else : 
        x = test_img
        labels = test_label.squeeze()
        print('\nIn test dataset ... ')
    
    for layer in model.values() :
        if isinstance(layer, softmax_with_crossEntropy) :
            x = layer.softmax_forward(x)
        else :
            x = layer.forward(x)
            
    preds = x.argmax(axis=1)
    acc = np.sum(np.where(preds==labels, True, False))/len(labels)
    return acc

print('\t Accuracy :', eval(model, train_version=True))
print('\t Accuracy :', eval(model, train_version=False))
print()

### regularization

In [ ]:
class Linear:
    def __init__(self, input_size=1, hidden_size=1):
        self.W = np.random.randn(input_size, hidden_size)
        self.b = np.zeros(hidden_size)
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x # 입력값 저장
        out = np.dot(x, self.W) + self.b # 선형 연산 수행
        return out

    def backward(self, dout, lr, lambda_reg=0.01):
        dx = np.dot(dout, self.W.T) # 입력값에 대한 미분값 계산
        self.dW = np.dot(self.x.T, dout) + lambda_reg * self.W # 가중치 행렬에 대한 미분값 계산
        self.db = np.sum(dout, axis=0) # bias에 대한 미분값 계산
        self.W -= lr * self.dW # 가중치 행렬 업데이트
        self.b -= lr * self.db  # bias 업데이트
        return dx


In [ ]:
class MLP_Classifier:
    def __init__(self):
        self.delta = 1e-7
        # softmax
        self.softmax_x = None
        self.softmax_out = None
        # crossEntropy
        self.pred = None
        self.target = None
        self.alpha = 0.001

    def softmax_forward(self, x):
        """
        Softmax forward propagation
        
        Parameters:
        - x (numpy array): Input data
        
        Returns:
        - softmax_out (numpy array): Output of softmax activation function
        """
        self.softmax_x = x
        # subtracting the maximum value for numerical stability
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        self.softmax_out = exp_x / np.sum(exp_x, axis=-1, keepdims=True)
        return self.softmax_out

    def crossEntropy_forward(self, pred, target):
        """
        Cross-entropy forward propagation
        
        Parameters:
        - pred (numpy array): Predicted output
        - target (numpy array): Target output
        
        Returns:
        - loss (float): Cross-entropy loss
        """
        self.pred = pred
        self.target = target
        # avoid log(0) by adding delta
        reg_term = 0.5 * self.alpha * np.sum(pred ** 2)
        loss = -np.sum(target * np.log(pred + self.delta)) / len(pred) + reg_term
        return loss

    def backward(self):
        """
        Backward propagation for cross-entropy and softmax
        
        Returns:
        - dx (numpy array): Gradient of loss with respect to the input
        """
        batch_size = self.pred.shape[0]
        dx = (self.pred - self.target) / batch_size
        return dx


In [ ]:

# train version 1

from collections import OrderedDict

def train_MLP(config) :
    lr, num_epoch,model = config['learning_rate'], config['num_epoch'], config['model']
    print_loss_interval = 1
    
    layer1 = Linear(784, 100)
    relu = Relu()
    layer2 = Linear(100, 10)
    softmax_with_CE = model
    
    loss_lst = list()
    iter_lst = list()
    weight_list = list()
    
    for iter in range(num_epoch) :
        # forward
        x = layer1.forward(train_img)
        x = relu.forward(x)
        x = layer2.forward(x)
        preds = softmax_with_CE.softmax_forward(x)
        
        # loss
        one_hot_labels = make_one_hot(train_label)
        losses = softmax_with_CE.crossEntropy_forward(preds, one_hot_labels)
        loss = losses.sum()/len(preds)
        
        if iter % print_loss_interval == 0:
            print("[epoch %d / %d] average loss : %f" % (iter, num_epoch, loss))
        
        loss_lst.append(loss)
        iter_lst.append(iter)
        
        # backward
        dL = softmax_with_CE.backward()
        weight_list.append(np.sum(dL))
        dL = layer2.backward(dL, lr)
        dL = relu.backward(dL)
        dL = layer1.backward(dL,lr)

        
    model = OrderedDict()
    model['layer1'] = layer1
    model['relu'] = relu
    model['layer2'] = layer2
    model['softmax_with_CE'] = softmax_with_CE
    
    return model, loss_lst, iter_lst, weight_list

In [ ]:
config = { 'model' : MLP_Classifier(),
            'learning_rate' : 0.1,
            'num_epoch' : 100
          }

model, loss_lst, iter_lst,weight_list = train_MLP(config)

In [ ]:
plt.plot(iter_lst, loss_lst)
plt.title("Loss vs. Iteration")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(Non_weight_lsit)
plt.title("Not applied in Regluarization")
plt.xlabel("Iteration")
plt.ylabel("Weight")
plt.show()

In [ ]:
plt.plot(weight_list)
plt.title("L2 Regularization")
plt.xlabel("Iteration")
plt.ylabel("Weight")
plt.show()

In [ ]:
def eval(model, train_version = True) :
    if train_version :
        x = train_img
        labels = train_label.squeeze()
        print('In train dataset ... ')
    else : 
        x = test_img
        labels = test_label.squeeze()
        print('\nIn test dataset ... ')
    
    for layer in model.values() :
        if isinstance(layer, MLP_Classifier) :
            x = layer.softmax_forward(x)
        else :
            x = layer.forward(x)
            
    preds = x.argmax(axis=1)
    acc = np.sum(np.where(preds==labels, True, False))/len(labels)
    return acc

print('\t Accuracy :', eval(model, train_version=True))
print('\t Accuracy :', eval(model, train_version=False))
print()

# 5

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

# install
## numpy
## matplotlib

data load & preprocessing

In [ ]:
from dataset.mnist import load_mnist

(train_raw_img, train_label), (test_raw_img, test_label) = load_mnist(flatten=False, normalize=False)
print(train_raw_img.shape)

In [ ]:
# preprocessing (train & inference)

train_img = train_raw_img.reshape(len(train_raw_img.squeeze()), -1)
train_label = train_label.reshape(len(train_label), -1)

test_img = test_raw_img.reshape(len(test_raw_img.squeeze()), -1)
test_label = test_label.reshape(len(test_label), -1)

print(train_img.shape)
print(train_label.shape)
print(test_img.shape)
print(test_label.shape)

In [ ]:
# normalization (set value 0 ~ 1)

train_img = train_img.astype('float')
train_img = train_img/255

test_img = test_img.astype('float')
test_img = test_img/255

model

In [ ]:

class Linear:
    def __init__(self, input_size=1, hidden_size=1):
        # Initialize weights and biases
        self.W = np.random.randn(input_size, hidden_size)
        self.b = np.zeros(hidden_size)
        
        # Store input size and hidden size
        self.input_size = input_size
        self.hidden_size = hidden_size
        
    def forward(self, x):
        # Calculate linear transformation
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout, lr = 0.01, lamb=0):
        # Calculate gradients for weights and biases
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        # Update weights and biases
        self.W -= lr * (dW + lamb*self.W)
        self.b -= lr * db
        
        # Calculate gradients for input
        # print(dout.shape, self.W.T.shape)
        dx = np.dot(dout, self.W.T)
        
        return dx


activation functions (ReLU, LeakyReLU, Sigmoid, Tanh)

In [ ]:
class ReLU:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
        
    def backward(self, dout):
        dx = dout
        dx[self.mask] = 0
        return dx
    

class LeakyReLU:
    def __init__(self, alpha=0.01):
        self.alpha = alpha
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] *= self.alpha
        return out
        
    def backward(self, dout):
        dout[self.mask] *= self.alpha
        dx = dout
        return dx

class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
        
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        return dx

class Tanh:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = np.tanh(x)
        self.out = out
        return out
        
    def backward(self, dout):
        dx = dout * (1 - self.out ** 2)
        return dx


In [ ]:
class Softmax_with_CrossEntropy :
    def __init__(self) :
        # softmax
        self.softmax_x = None 
        self.softmax_out = None
        # crossEntropy
        self.pred = None
        self.target = None
        self.delta = 1e-7
        
    def softmax_forward(self, x) :
        self.softmax_x = x
        tmp = np.max(self.softmax_x, axis=1).reshape(-1, 1)
        self.softmax_out = np.exp(self.softmax_x-tmp)/np.sum(np.exp(self.softmax_x-tmp), axis=1).reshape(-1,1)
        return self.softmax_out
    
    def crossEntropy_forward(self, pred, target) :
        self.pred = pred
        self.target = target
        loss = -np.sum(self.target*np.log(self.pred+self.delta), axis=1)
        return loss
    
    def backward(self) :
        dout = (self.pred-self.target)/len(self.pred)
        return dout

## Train / Eval

In [ ]:
# one_hot label 만드는 함수

def make_one_hot(labels) :
    a = []
    for label in labels :
        one_hot = np.zeros(10)
        one_hot[label] = 1
        a.append(one_hot)
    a = np.array(a)
    return a

# one_hot_labels = make_one_hot(train_label)
# print(train_label[0])
# print(one_hot_labels[0])

In [ ]:
def eval(model, train_version = True) :
    if train_version :
        x = train_img
        labels = train_label.squeeze()
        print('In train dataset ... ')
    else : 
        x = test_img
        labels = test_label.squeeze()
        print('\nIn test dataset ... ')
    
    for layer in model.values() :
        if isinstance(layer, Softmax_with_CrossEntropy) :
            x = layer.softmax_forward(x)
        else :
            x = layer.forward(x)
            
    preds = x.argmax(axis=1)
    acc = np.sum(np.where(preds==labels, True, False))/len(labels)
    return acc

In [ ]:
from collections import OrderedDict

def train_MLP(config) :
    lr, num_epoch = config['learning_rate'], config['num_epoch']
    print_loss_interval = 20
    
    layer1 = Linear(784, 100)
    activation_function = config['activation_function']
    layer2 = Linear(100, 10)
    softmax_with_CE = Softmax_with_CrossEntropy()
    
    for iter in range(num_epoch) :
        # forward
        x = layer1.forward(train_img)
        x = activation_function.forward(x)
        x = layer2.forward(x)
        preds = softmax_with_CE.softmax_forward(x)
        
        # loss
        one_hot_labels = make_one_hot(train_label)
        losses = softmax_with_CE.crossEntropy_forward(preds, one_hot_labels)
        loss = losses.sum()/len(preds)
        
        # backward
        dL = softmax_with_CE.backward()
        dL = layer2.backward(dL, lr)
        dL = activation_function.backward(dL)
        dL = layer1.backward(dL, lr)
        
        if iter == 0 or (iter+1) % print_loss_interval == 0:
            print("[epoch %d / %d] average loss : %f" % (iter+1, num_epoch, loss))
            
    model = OrderedDict()
    model['layer1'] = layer1
    model['activation_function'] = activation_function
    model['layer2'] = layer2
    model['softmax_with_CE'] = softmax_with_CE
    
    return model

In [ ]:
config1 = { 'learning_rate' : 0.1,
            'num_epoch' : 100,
            'activation_function' : ReLU()
          }

config2 = { 'learning_rate' : 0.1,
            'num_epoch' : 100,
            'activation_function' : LeakyReLU()
          }

config3 = { 'learning_rate' : 0.1,
            'num_epoch' : 100,
            'activation_function' : Sigmoid()
          }

config4 = { 'learning_rate' : 0.1,
            'num_epoch' : 100,
            'activation_function' : Tanh()
          }

print('---ReLU---')
model1 = train_MLP(config1)

print('\n---Leaky_Relu---')
model2 = train_MLP(config2)

print('\n---Sigmoid---')
model3 = train_MLP(config3)

print('\n---TanH---')
model4 = train_MLP(config4)

In [ ]:
# evaluation

print('\t Accuracy :', eval(model1, train_version=False))
print('\t Accuracy :', eval(model2, train_version=False))
print('\t Accuracy :', eval(model3, train_version=False))
print('\t Accuracy :', eval(model4, train_version=False))

type of gradient descent

In [ ]:
class Softmax_with_CrossEntropy :
    def __init__(self) :
        # softmax
        self.softmax_x = None 
        self.softmax_out = None
        # crossEntropy
        self.pred = None
        self.target = None
        self.delta = 1e-7
        
    def softmax_forward(self, x) :
        self.softmax_x = x
        tmp = np.max(self.softmax_x, axis=1).reshape(-1, 1)
        self.softmax_out = np.exp(self.softmax_x-tmp)/np.sum(np.exp(self.softmax_x-tmp), axis=1).reshape(-1,1)
        return self.softmax_out
    
    def crossEntropy_forward(self, pred, target) :
        self.pred = pred
        self.target = target
        loss = -np.sum(self.target*np.log(self.pred+self.delta), axis=1)
        return loss
    
    def backward(self) :
        dout = (self.pred-self.target)/len(self.pred)
        return dout

In [ ]:
from collections import OrderedDict

def train_MLP_v2(config,train_img = train_img, train_label = train_label) :
    lr, num_epoch, batch_size = config['learning_rate'], config['num_epoch'], config['batch_size']
    print_loss_interval = 20
    
    # Initialize the layers of the MLP
    layer1 = Linear(784, 100) # First linear layer with 784 input features and 100 output features
    activation_function = config['activation_function'] # Activation function object, such as ReLU
    layer2 = Linear(100, 10) # Second linear layer with 100 input features and 10 output features
    softmax_with_CE = Softmax_with_CrossEntropy() # Softmax with Cross-Entropy loss function object
    
    for iter in range(num_epoch):
        # Forward and backward propagation for each mini-batch
        num_data = len(train_img)
        num_batch = num_data // batch_size
        if num_data % batch_size != 0:
            num_batch += 1

        epoch_loss = 0

        for batch_idx in range(num_batch):
            start_idx = batch_idx * batch_size
            end_idx = min(num_data, (batch_idx + 1) * batch_size)

            batch_img = train_img[start_idx:end_idx]
            batch_label = train_label[start_idx:end_idx]

            x = layer1.forward(batch_img) # Forward propagation through the first linear layer
            x = activation_function.forward(x) # Forward propagation through the activation function
            x = layer2.forward(x) # Forward propagation through the second linear layer
            preds = softmax_with_CE.softmax_forward(x) # Calculate the softmax probabilities

            # Calculate the loss and the gradient of the loss w.r.t the predictions
            one_hot_labels = make_one_hot(batch_label)
            losses = softmax_with_CE.crossEntropy_forward(preds, one_hot_labels)
            batch_loss = losses.sum() / len(preds)
            epoch_loss += batch_loss
            dL = softmax_with_CE.backward() # Backward propagation through the loss function
            dL = layer2.backward(dL, lr) # Backward propagation through the second linear layer
            dL = activation_function.backward(dL) # Backward propagation through the activation function
            dL = layer1.backward(dL, lr) # Backward propagation through the first linear layer

        avg_loss = epoch_loss / num_batch

        # Print the average loss at the current epoch
        if iter == 0 or (iter + 1) % print_loss_interval == 0:
            print("[epoch %d / %d] average loss : %f" % (iter + 1, num_epoch, avg_loss))

    # Save the trained model as an ordered dictionary
    model = OrderedDict()
    model['layer1'] = layer1
    model['activation_function'] = activation_function
    model['layer2'] = layer2
    model['softmax_with_CE'] = softmax_with_CE
    
    return model


In [ ]:
print('---batch---')
config_batch = { 'learning_rate' : 0.1,
            'num_epoch' : 20,
            'activation_function' : ReLU(),
            'batch_size' : len(train_img) 
          }
model_batch = train_MLP_v2(config_batch)


print('\n---mini_batch---')
config_mini_batch = { 'learning_rate' : 0.1,
            'num_epoch' : 20,
            'activation_function' : ReLU(),
            'batch_size' : 2500
          }
model_mini_batch = train_MLP_v2(config_mini_batch)

print('\n---stochastic---')
config_stochastic = { 'learning_rate' : 0.001,
            'num_epoch' : 20,
            'activation_function' : ReLU(),
            'batch_size' : 1
          }
model_stochastic = train_MLP_v2(config_stochastic)

In [ ]:
print('\t Accuracy :', eval(model_batch, train_version=False))
print('\t Accuracy :', eval(model_mini_batch, train_version=False))
print('\t Accuracy :', eval(model_stochastic, train_version=False))

(Multi Support Vector Machine)

In [ ]:

class Linear_v2:
    def __init__(self, input_size=1, hidden_size=1):
        # Initialize weights and biases
        self.W = np.random.randn(input_size, hidden_size)
        self.b = np.zeros(hidden_size)
        
        # Store input size and hidden size
        self.input_size = input_size
        self.hidden_size = hidden_size
        
    def forward(self, x):
        # Calculate linear transformation
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout, lr, lamb=0):
        # Calculate gradients for weights and biases    
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        dW += lamb * self.W
        # Update weights and biases
        self.W -= lr * dW
        self.b -= lr * db
        
        # Calculate gradients for input
        # print(dout.shape, self.W.T.shape)
        dx = np.dot(dout, self.W.T)
    
        
        return dx

In [ ]:
class Softmax_with_MSVM:
    def __init__(self):
        # softmax
        self.softmax_out = None
        self.softmax_x = None
        # MSVM
        self.pred = None
        self.target = None
        self.reg = None

    def softmax_forward(self, x):
        self.softmax_x = x
        # subtracting the maximum value for numerical stability
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        self.softmax_out = exp_x / np.sum(exp_x, axis=-1, keepdims=True)
        return self.softmax_out
    
    def sigmoid_forward(self, x):
        self.sigmoid_x = x
        self.sigmoid_out = 1 / (1 + np.exp(-x))
        return self.sigmoid_out
        

    def forward(self, x, target, reg=0.1):
        self.target = target
        self.MSVM_out = self.softmax_forward(x)
        scores = np.dot(x, self.MSVM_out.T)
        correct_scores = scores[np.arange(x.shape[0]), target.flatten()]
        margins = np.maximum(0, scores - correct_scores.reshape(-1, 1) + 1)
        margins[np.arange(x.shape[0]), target.flatten()] = 0
        loss = np.sum(margins)
        loss /= x.shape[0]
        loss += 0.5 * reg * np.sum(self.MSVM_out * self.MSVM_out)
        self.pred = (scores >= np.max(scores, axis=1).reshape(-1, 1)).astype(float)
        self.margin = margins
        return loss

    def backward(self, lr, reg):
        self.reg = reg
        batch_size = self.pred.shape[0]
        dL_dsoftmax = self.MSVM_out - self.target

        dsoftmax_dMSVMx = (self.softmax_out * (1 - self.softmax_out))
        dL_dMSVMx = dL_dsoftmax * dsoftmax_dMSVMx

        zero_mask = (dL_dsoftmax == 0) | (self.MSVM_out == 0)
        dL_dMSVMx = np.where(zero_mask, 0, -dL_dMSVMx)
        dMSVMx_dw = np.dot(dL_dMSVMx.T, self.pred) / batch_size

        # L2 regularization
        dMSVMx_dw += self.reg * self.MSVM_out

        self.MSVM_out -= np.clip(lr * dMSVMx_dw.T, -1e8, 1e8)
        return dL_dMSVMx



In [ ]:

from collections import OrderedDict

def clip_gradient(grad, clip_value):
    return np.clip(grad, -clip_value, clip_value)

def train_MSVM(config, train_img=train_img, train_label=train_label):
    lr, num_epoch, batch_size, reg, grad_clip = config['learning_rate'], config['num_epoch'], config['batch_size'], config['reg'], config['grad_clip']
    print_loss_interval = 20
    
    layer1 = Linear_v2(784, 100)
    activation_function = config['activation_function']
    layer2 = Linear_v2(100, 10)
    softmax_with_MSVM = Softmax_with_MSVM()
    
    for iter in range(num_epoch):
        # forward
        num_data = len(train_img)
        num_batch = num_data // batch_size
        if num_data % batch_size != 0:
            num_batch += 1

        epoch_loss = 0

        for batch_idx in range(num_batch):
            start_idx = batch_idx * batch_size
            end_idx = min(num_data, (batch_idx + 1) * batch_size)

            batch_img = train_img[start_idx:end_idx]
            batch_label = train_label[start_idx:end_idx]

            x = layer1.forward(batch_img)
            x = activation_function.forward(x)
            x = layer2.forward(x)
            preds = softmax_with_MSVM.softmax_forward(x)
            
            loss = softmax_with_MSVM.forward(preds, batch_label, config['reg'])
            batch_loss = loss.sum() / len(preds)
            epoch_loss += batch_loss

            # backward
            dL = softmax_with_MSVM.backward(lr, reg)
            dL = clip_gradient(dL, grad_clip)
            dL = layer2.backward(dL, lr)
            dL = clip_gradient(dL, grad_clip)
            dL = activation_function.backward(dL)
            dL = clip_gradient(dL, grad_clip)
            dL = layer1.backward(dL, lr)
            dL = clip_gradient(dL, grad_clip)

        avg_loss = epoch_loss / num_batch
        print(avg_loss)

        if iter == 0 or (iter + 1) % print_loss_interval == 0:
            print("[epoch %d / %d] average loss : %f" % (iter + 1, num_epoch, avg_loss))

    model = OrderedDict()
    model['layer1'] = layer1
    model['activation_function'] = activation_function
    model['layer2'] = layer2
    model['softmax_with_MSVM'] = softmax_with_MSVM
    
    return model


In [ ]:
config = { 'learning_rate' : 0.0001,
            'num_epoch' : 20,
            'activation_function' : ReLU(),
            'batch_size' : 10,
            'reg' : 0.1,
            'grad_clip' : 5
          }


model = train_MSVM(config)

In [ ]:
def eval(model, train_version = True) :
    if train_version :
        x = train_img
        labels = train_label.squeeze()
        print('In train dataset ... ')
    else : 
        x = test_img
        labels = test_label.squeeze()
        print('\nIn test dataset ... ')
    
    for layer in model.values() :
        if isinstance(layer, Softmax_with_MSVM) :
            x = layer.softmax_forward(x)
        else :
            x = layer.forward(x)
            
    preds = x.argmax(axis=1)
    acc = np.sum(np.where(preds==labels, True, False))/len(labels)
    return acc

In [ ]:
print('\t Accuracy :', eval(model, train_version=False))

 (3-layer 이상 MLP 구현)

In [ ]:
class ReLU:
    def __init__(self):
        self.mask = None
        
    def get_mask(self):
        return self.mask.shape
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
        
    def backward(self, dout, x):
        if self.mask.shape != x.shape:
            self.mask = np.zeros_like(x, dtype = np.bool)
        dx = dout
        dx[self.mask] = 0
        return dx 

    
class LeakyReLU:
    def __init__(self, alpha=0.01):
        self.alpha = alpha
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] *= self.alpha
        return out
    
    def backward(self, dout, x):
        if self.mask is None or self.mask.shape != x.shape:
            self.mask = np.zeros_like(x, dtype=np.bool)
        dx = dout
        dx[self.mask] *= self.alpha
        return dx


class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
        
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        return dx

class Tanh:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = np.tanh(x)
        self.out = out
        return out
        
    def backward(self, dout):
        dx = dout * (1 - self.out ** 2)
        return dx


In [ ]:
class Softmax_with_Mul_CrossEntropy:
    def __init__(self):
        self.softmax_x = None 
        self.softmax_out = None
        self.pred = None
        self.target = None

    def softmax_forward(self, x):
        self.softmax_x = x
        tmp = np.max(self.softmax_x, axis=1).reshape(-1, 1)
        self.softmax_out = np.exp(self.softmax_x - tmp) / np.sum(np.exp(self.softmax_x - tmp), axis=1).reshape(-1, 1)
        return self.softmax_out
    
    def crossEntropy_forward(self, pred, target):
        delta = 1e-7
        self.pred = pred
        self.target = target
        loss = -np.sum(self.target * np.log(self.pred + delta), axis=1)
        return loss
    
    def backward(self, dL_next):
        batch_size = len(self.pred)
        dsoftmax = (self.pred - self.target) / batch_size
        dL = dsoftmax * dL_next
        return dL


In [ ]:
from collections import OrderedDict

def train_Multi_MLP(config):
    lr, num_epoch, batch_size = config['learning_rate'], config['num_epoch'], config['batch_size']
    print_loss_interval = 1
    
    layer1 = Linear(784, 256)
    activation_function1 = config['activation_function']
    layer2 = Linear(256, 128)
    activation_function2 = config['activation_function']
    layer3 = Linear(128, 64)
    activation_function3 = config['activation_function']
    layer4 = Linear(64, 32)
    activation_function4 = config['activation_function']
    layer5 = Linear(32, 10)
    softmax_with_CE = Softmax_with_Mul_CrossEntropy()
    
    for iter in range(num_epoch):
        total_loss = 0
        num_batches = 0
        
        for i in range(0, len(train_img), batch_size):
            # forward
            batch_img = train_img[i:i+batch_size]
            batch_label = train_label[i:i+batch_size]
            
            x = layer1.forward(batch_img)
            x_1 = activation_function1.forward(x)
            x = layer2.forward(x_1)
            x_2 = activation_function2.forward(x)
            x = layer3.forward(x_2)
            x_3 = activation_function3.forward(x)
            x = layer4.forward(x_3)
            x_4 = activation_function4.forward(x)
            x = layer5.forward(x_4)
            preds = softmax_with_CE.softmax_forward(x)

            # loss
            one_hot_labels = make_one_hot(batch_label)
            losses = softmax_with_CE.crossEntropy_forward(preds, one_hot_labels)
            batch_loss = losses.sum() / len(preds)
            total_loss += batch_loss
            num_batches += 1

            # backward
            dL = softmax_with_CE.backward(one_hot_labels)
            dL = layer5.backward(dL)
            dL = activation_function4.backward(dL,x_4)
            dL = layer4.backward(dL)        
            dL = activation_function3.backward(dL,x_3)
            dL = layer3.backward(dL)
            dL = activation_function2.backward(dL,x_2)
            dL = layer2.backward(dL)
            dL = activation_function1.backward(dL,x_1)
            dL = layer1.backward(dL)

        # average batch loss
        avg_loss = total_loss / num_batches
        
        if iter == 0 or (iter+1) % print_loss_interval == 0:
            print("[epoch %d / %d] average loss : %f" % (iter+1, num_epoch, avg_loss))
            
    model = OrderedDict()
    model['layer1'] = layer1
    model['activation_function1'] = activation_function1
    model['layer2'] = layer2
    model['activation_function2'] = activation_function2
    model['layer3'] = layer3
    model['activation_function3'] = activation_function3
    model['layer4'] = layer4
    model['activation_function4'] = activation_function4
    model['layer5'] = layer5
    model['softmax_with_CE'] = softmax_with_CE
    
    return model


In [ ]:
config = { 'learning_rate' : 0.00001,
            'num_epoch' : 3,
            'activation_function' : LeakyReLU(),
            'batch_size' : 2500,
            'reg' : 0.1
          }

model = train_Multi_MLP(config)